In [1]:
# Instala o pacote tweepy
#!pip install tweepy

## IMPORTS

In [1]:
import tweepy as tw
import time as tm
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

## FUNCTIONS

In [2]:
def create_db(name):
    # cria o bd, a table registros e retorna o con e o cur
    name = name + '.db'
    con = sqlite3.connect(name)
    cur = con.cursor()
    cur.execute('CREATE TABLE registros (texto TEXT)')
    return con, cur

In [3]:
def connect_twitter(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
    # conectando ao twitter
    return tw.Client(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    

In [4]:
def extracting_data(client_twitter, query, start_time, end_minutes_time):
    """
    client_twitter : obj retornado da função connect_twitter
    query : termo para busca
    start_time : obj do tipo datetime
    end_time : int com minutos
    """
    result = []
    minutes = 1
    while minutes <= end_minutes_time:
        end_time = start_time + timedelta(days = 0, minutes = minutes)
        print(end_time)
        res = client_twitter.search_recent_tweets(query=query, max_results=100, start_time=start_time, end_time=end_time)
        res = res.data
        result.append(res)
        tm.sleep(5)
        minutes +=1
    return result

In [5]:
def insert_db(cur, con, data):
    for i in data:
        for x in i:
        #text = x
        #print(x.text)
            cur.execute("INSERT INTO registros (texto) VALUES (?);", (x.text,))
        #tm.sleep(1)
    con.commit()
    con.close()

In [5]:
def read_data(name_db):
    # conectando ao banco sqlite
    db = name_db + '.db'
    con = sqlite3.connect(db)
    cur = con.cursor()
    # busca todos os registros
    cur.execute('SELECT * FROM registros')
    res = cur.fetchall()
    con.close()
    #adiciona a um df
    res = pd.DataFrame(res)
    res.columns = ['Text']
    return res

In [7]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
BEARER_TOKEN = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [8]:
# criando conexão a api do twitter
client = connect_twitter(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [9]:
# criando BD para adicionar os dados
con, cur = create_db('db_war')

In [10]:
star = datetime(year=2022, month=3, day=1, hour=12, minute=15, second=0)
end_minutes_time = 10
query = 'war'
aux = extracting_data(client_twitter=client, query=query, start_time=star, end_minutes_time=end_minutes_time)

2022-03-01 12:16:00
2022-03-01 12:17:00
2022-03-01 12:18:00
2022-03-01 12:19:00
2022-03-01 12:20:00
2022-03-01 12:21:00
2022-03-01 12:22:00
2022-03-01 12:23:00
2022-03-01 12:24:00
2022-03-01 12:25:00


In [13]:
aux[0]

[<Tweet id=1498633146140704769 text='RT @Mandoline_Blue: The European parliament will call for EU-wide restrictions on imports of Russian oil &amp; gas, to strategically weaken Rus…'>,
 <Tweet id=1498633145914359816 text='RT @ByRakeshSimha: ▶️If you supported the savage bombing of Baghdad and Basra\n▶️If supported the war that led to the deaths of a million Ir…'>,
 <Tweet id=1498633145855447045 text="@CorbynOver @AaronBastani A war specifically between 2 countries cannot be classified as a World War. This shouldn't really need explaining.">,
 <Tweet id=1498633145628999682 text='RT @piersmorgan: There’s a 40-MILE convoy of Russian military vehicles heading towards Kyiv to surround the city, trap the people there, an…'>,
 <Tweet id=1498633145620602880 text='RT @CalltoActivism: BREAKING: the International Criminal Court will open an inquiry  into Russia’s invasion of Ukraine.\n\nRT if you want the…'>,
 <Tweet id=1498633145540755460 text='RT @Sagar08573388: one indian name Naveen lost his 

In [11]:
insert_db_test(cur, con, aux)

In [14]:
df = read_data('db_war')
df.head()

,Text
0,RT @Mandoline_Blue: The European parliament wi...
1,RT @ByRakeshSimha: ▶️If you supported the sava...
2,@CorbynOver @AaronBastani A war specifically b...
3,RT @piersmorgan: There’s a 40-MILE convoy of R...
4,RT @CalltoActivism: BREAKING: the Internationa...


In [15]:
df.to_csv('base_war_twitter.csv')